##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# CSV 데이터 로드

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/csv"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/csv.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/csv.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/load_data/csv.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a></td>
</table>

이 튜토리얼은 파일에서 `tf.data.Dataset`로 CSV 데이터를 로드하는 방법의 예를 제공합니다.

이 튜토리얼에서 사용된 데이터는 Titanic 승객 목록에서 가져온 것입니다. 이 모델은 연령, 성별, 티켓 등급 및 단독 여행 여부와 같은 특성을 기반으로 승객의 생존 가능성을 예측합니다.

## 설정

In [ ]:
import functools

import numpy as np
import tensorflow as tf

In [ ]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [ ]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

## 데이터 로드하기

시작하려면 CSV 파일의 상단을 보고 형식이 어떻게 지정되는지 봅니다.

In [ ]:
!head {train_file_path}

[pandas를 사용하여 로드](pandas_dataframe.ipynb)하고 NumPy 배열을 TensorFlow에 전달할 수 있습니다. 큰 파일 세트로 확장해야 하거나 [TensorFlow 및 tf.data](../../guide/data.ipynb)와 통합되는 로더가 필요한 경우, `tf.data.experimental.make_csv_dataset` 함수를 사용합니다.

명시적으로 식별해야 하는 유일한 열은 모델에서 예측하려는 값을 가진 열입니다. 

In [ ]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

이제 파일에서 CSV 데이터를 읽고 데이터세트를 작성합니다.

(전체 설명서는 `tf.data.experimental.make_csv_dataset`를 참조하세요.)


In [ ]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [ ]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

데이터세트의 각 항목은 배치이며 (*많은 예제*, *많은 레이블* )의 튜플로 표현됩니다. 예제의 데이터는 행 기반 텐서가 아닌 열 기반 텐서로 구성되며, 각 데이터는 배치 크기(이 경우 5)만큼 많은 요소가 있습니다.

직접 보는 것이 도움이 될 수 있습니다.

In [ ]:
show_batch(raw_train_data)

보시다시피 CSV의 열 이름이 지정됩니다. 데이터세트 생성자가 이들 이름을 자동으로 선택합니다. 작업 중인 파일의 첫 번째 줄에 열 이름이 포함되어 있지 않은 경우, 열 이름을 문자열 목록으로 `make_csv_dataset` 함수의 `column_names` 인수로 전달합니다.

In [ ]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

이 예제에서는 사용 가능한 모든 열을 사용합니다. 데이터세트에서 일부 열을 생략해야 하는 경우, 사용하려는 열의 목록만 작성하고 생성자의 (선택적) `select_columns` 인수로 전달합니다.


In [ ]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

## 데이터 전처리

CSV 파일은 다양한 데이터 유형을 포함할 수 있습니다. 일반적으로 데이터를 모델에 공급하기 전에 혼합 유형에서 고정 길이 벡터로 변환하려고 합니다.

TensorFlow에는 일반적인 입력 변환을 설명하기 위한 내장 시스템이 있습니다. 자세한 내용은 `tf.feature_column`, [이 튜토리얼](../keras/feature_columns)을 참조하세요.

원하는 도구(예: [nltk](https://www.nltk.org/) 또는 [sklearn](https://scikit-learn.org/stable/))를 사용하여 데이터를 전처리하고 처리된 출력을 TensorFlow에 전달하면 됩니다.

모델 내에서 전처리를 수행할 때의 주요 이점은 모델을 내보낼 때 전처리가 포함된다는 것입니다. 이렇게 하면 원시 데이터를 모델로 직접 전달할 수 있습니다.

### 연속 데이터

데이터가 이미 적절한 숫자 형식인 경우, 데이터를 모델로 전달하기 전에 벡터로 묶을 수 있습니다.

In [ ]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

In [ ]:
example_batch, labels_batch = next(iter(temp_dataset)) 

다음은 모든 열을 묶는 간단한 함수입니다.

In [ ]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

이 함수를 데이터세트의 각 요소에 적용합니다.

In [ ]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

혼합 데이터 유형이 있는 경우, 해당 단순 숫자 필드를 분리할 수 있습니다. `tf.feature_column` API로 처리할 수 있지만, 약간의 오버헤드가 발생하며 실제로 필요하지 않으면 피해야 합니다. 혼합 데이터세트로 다시 전환합니다.

In [ ]:
show_batch(raw_train_data)

In [ ]:
example_batch, labels_batch = next(iter(temp_dataset)) 

따라서 숫자 특성 목록을 선택하고 단일 열로 묶는 보다 일반적인 전처리기를 정의합니다.

In [ ]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [ ]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [ ]:
show_batch(packed_train_data)

In [ ]:
example_batch, labels_batch = next(iter(packed_train_data)) 

#### 데이터 정규화

연속 데이터는 항상 정규화되어야 합니다.

In [ ]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

In [ ]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [ ]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std


이제 숫자 열을 만듭니다. `tf.feature_columns.numeric_column` API는 각 배치에서 실행될 `normalizer_fn` 인수를 허용합니다.

[`functools.partial`](https://docs.python.org/3/library/functools.html#functools.partial)를 사용하여 `MEAN` 및 `STD`를 노멀라이저 fn에 바인딩합니다.

In [ ]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

모델을 훈련할 때 이 특성 열을 포함하여 이 숫자 데이터 블록을 선택하고 중앙에 배치합니다.

In [ ]:
example_batch['numeric']

In [ ]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

여기에 사용된 평균 기반 정규화를 위해서는 각 열의 평균을 미리 알아야 합니다.

### 범주형 데이터

CSV 데이터의 일부 열은 범주형 열입니다. 즉, 콘텐츠는 제한된 옵션 세트 중 하나여야 합니다.

`tf.feature_column` API를 사용하여 각 범주 열에 대해 `tf.feature_column.indicator_column`을 가진 모음을 작성합니다.


In [ ]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}


In [ ]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [ ]:
# See what you just created.
categorical_columns

In [ ]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

이것은 나중에 모델을 빌드할 때 데이터 처리 입력의 일부가 됩니다.

### 결합된 전처리 레이어

두 개의 특성 열 모음을 추가하고 `tf.keras.layers.DenseFeatures`에 전달하여 두 입력 유형을 추출하고 전처리할 입력 레이어를 만듭니다.

In [ ]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [ ]:
print(preprocessing_layer(example_batch).numpy()[0])

## 모델 빌드하기

`preprocessing_layer`를 사용하여 `tf.keras.Sequential`를 빌드합니다.

In [ ]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

## 훈련, 평가 및 예측하기

이제 모델을 인스턴스화하고 훈련할 수 있습니다.

In [ ]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [ ]:
model.fit(train_data, epochs=20)

모델이 학습하면 `test_data` 세트에서 정확성을 확인할 수 있습니다.

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

`tf.keras.Model.predict`를 사용하여 배치 또는 배치 데이터세트에서 레이블을 유추합니다.

In [ ]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))
